In [46]:
from sylliba.api.subnet_api import SubnetAPI
from sylliba.validator import reward_text, reward_speech
from sylliba.protocol import TranslateRequest
from modules.translation.data_models import TranslationRequest
from sylliba.utils.uids import get_top_miner_uids
import bittensor as bt
from neurons.enums.models import *
from importlib import import_module
from neurons.utils.serialization import audio_encode, audio_decode
import numpy as np

In [47]:
request = {
    "input": "Hi How are you doing today?",
    "task_string": "text2speech",
    "source_language": "English",
    "target_language": "French"
}

In [48]:
translation_synapse = TranslateRequest(translation_request = TranslationRequest(data = request))

In [49]:
wallet = bt.wallet('sylliba', 'validator1', '~/.bittensor/wallets/')

In [ ]:
subtensor = bt.subtensor(network='test')
metagraph = subtensor.metagraph(netuid = 197)

miner_uids = await get_top_miner_uids(metagraph, wallet)
miner_axons = [metagraph.axons[uid] for uid in miner_uids]

print(miner_uids)

In [ ]:
subnet_api = SubnetAPI(wallet=wallet)

responses = await subnet_api(
    axons=miner_axons,
    synapse=translation_synapse,
    timeout=5
)

print(responses)

In [52]:
def process_validator_output(miner_response, input_string, task_string):
    if task_string.endswith('text'):
        scores = [reward_text(miner_response, input_string, import_module(llm)) for llm in LLMS]
    else:
        scores = [reward_speech(miner_response, input_string, import_module(llm)) for llm in LLMS]
    return sum(scores) / len(scores)

In [ ]:
task_string = request['task_string']
results = []

for j in range(0, len(responses)):
    if responses[j].miner_response is not None:
        if task_string.endswith('speech'):
            miner_output_data = audio_decode(responses[j].miner_response)
        else:
            miner_output_data = responses[j].miner_response
                    
        results.append(
            float(process_validator_output(
                miner_output_data,
                request['input'],
                task_string
            )) # 'numpy.float64' object cannot be interpreted as integer
        )

print(results)

In [ ]:
best_response = responses[np.argmax(results)]
print(best_response)